In [1]:
import torch 
from torch import nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import datasets
import torchvision.transforms as transforms
import os,PIL,pathlib
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

#设置GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [2]:
data_dir = './data/bird_photos'
data_dir = pathlib.Path(data_dir)

In [3]:
image_count = len(list(data_dir.glob('*/*')))
image_count # 这里输出的是图片的总数

566

In [4]:
data_dir = './data/bird_photos'
data_transform = transforms.Compose([
    transforms.Resize([224,224]),
    transforms.ToTensor()
])

In [5]:
total_data = datasets.ImageFolder(data_dir,transform=data_transform)
total_data

Dataset ImageFolder
    Number of datapoints: 566
    Root location: ./data/bird_photos
    StandardTransform
Transform: Compose(
               Resize(size=[224, 224], interpolation=PIL.Image.BILINEAR)
               ToTensor()
           )

In [6]:
train_size = int(0.8*len(total_data))
test_size = len(total_data) - train_size
train_data,test_data = torch.utils.data.random_split(total_data,[train_size,test_size])

In [7]:
batch_size = 8
train_dl = torch.utils.data.DataLoader(train_data,batch_size=batch_size,shuffle=True)
test_dl = torch.utils.data.DataLoader(test_data,batch_size=batch_size)

In [9]:
class BasicConv2d(nn.Module):
    def __init__(self, in_channel, out_channel, **kwargs):
        super(BasicConv2d, self).__init__()
        self.conv = nn.Conv2d(in_channel, out_channel, bias=False, **kwargs)
        self.norm = nn.BatchNorm2d(out_channel, eps=0.001)
        self.relu = nn.ReLU(inplace=True)
        
    def forward(self, x):
        x = self.conv(x)
        x = self.norm(x)
        x = self.relu(x)
        return x

In [10]:
class InceptionA(nn.Module):
    def __init__(self, in_channel, pool_feature):
        super(InceptionA, self).__init__()
        # 1x1 conv
        self.conv1 = BasicConv2d(in_channel, 64, kernel_size=1)
        # 1x1 conv -> 5x5 conv
        self.conv2 = nn.Sequential(
            BasicConv2d(in_channel, 48, kernel_size=1),
            BasicConv2d(48, 64, kernel_size=5, padding=2)
        )
        # 1x1 conv -> [3x3 conv]x2
        self.conv3 = nn.Sequential(
            BasicConv2d(in_channel, 64, kernel_size=1),
            BasicConv2d(64, 96, kernel_size=3, padding=1),
            BasicConv2d(96, 96, kernel_size=3, padding=1)
        )
        # 3x3 avgpool -> 1x1 conv
        self.conv4 = nn.Sequential(
            nn.AvgPool2d(kernel_size=3, stride=1, padding=1),
            BasicConv2d(in_channel, pool_feature, kernel_size=1)
        )
        
    def forward(self, x):
        x1 = self.conv1(x)
        x2 = self.conv2(x)
        x3 = self.conv3(x)
        x4 = self.conv4(x)
        x = torch.cat([x1,x2,x3,x4], 1)
        return x

In [24]:
class InceptionB(nn.Module):
    def __init__(self, in_channel, filter7x7):
        super(InceptionB, self).__init__()
        # 1x1 conv
        self.conv1 = BasicConv2d(in_channel, 192, kernel_size=1)
        # 1x1 conv -> 1x7 conv -> 7x1 conv
        self.conv2 = nn.Sequential(
            BasicConv2d(in_channel, filter7x7, kernel_size=1),
            BasicConv2d(filter7x7, filter7x7, kernel_size=(1,7), padding=(0,3)),
            BasicConv2d(filter7x7, 192, kernel_size=(7,1), padding=(3,0))
        )
        # 1x1 conv -> [7x1 conv -> 1x7 conv]x2
        self.conv3 = nn.Sequential(
            BasicConv2d(in_channel, filter7x7, kernel_size=1),
            BasicConv2d(filter7x7, filter7x7, kernel_size=(7,1), padding=(3,0)),
            BasicConv2d(filter7x7, filter7x7, kernel_size=(1,7), padding=(0,3)),
            BasicConv2d(filter7x7, filter7x7, kernel_size=(7,1), padding=(3,0)),
            BasicConv2d(filter7x7, 192, kernel_size=(1,7), padding=(0,3))
        )
        # 3x3 avgpool -> 1x1 conv
        self.conv4 = nn.Sequential(
            nn.AvgPool2d(kernel_size=3, stride=1, padding=1),
            BasicConv2d(in_channel, 192, kernel_size=1)
        )
        
    def forward(self, x):
        x1 = self.conv1(x)
        x2 = self.conv2(x)
        x3 = self.conv3(x)
        x4 = self.conv4(x)
        x = torch.cat([x1,x2,x3,x4], 1)
        return x

In [25]:
class InceptionC(nn.Module):
    def __init__(self, in_channel):
        super(InceptionC, self).__init__()
        # 1x1 conv
        self.conv1 = BasicConv2d(in_channel, 320, kernel_size=1)
        # 1x1 conv -> [1x3 conv, 3x1 conv]
        self.conv2 = BasicConv2d(in_channel, 384, kernel_size=1)
        self.conv2a = BasicConv2d(384, 384, kernel_size=(1,3), padding=(0,1))
        self.conv2b = BasicConv2d(384, 384, kernel_size=(3,1), padding=(1,0))
        # 1x1 conv -> [3x3 conv]x2
        self.conv3 = nn.Sequential(
            BasicConv2d(in_channel, 448, kernel_size=1),
            BasicConv2d(448, 384, kernel_size=3, padding=1)
        )
        self.conv3a = BasicConv2d(384, 384, kernel_size=(1,3), padding=(0,1))
        self.conv3b = BasicConv2d(384, 384, kernel_size=(3,1), padding=(1,0))
        # 3x3 avgpool -> 1x1 conv
        self.conv4 = nn.Sequential(
            nn.AvgPool2d(kernel_size=3, stride=1, padding=1),
            BasicConv2d(in_channel, 192, kernel_size=1)
        )
        
    def forward(self, x):
        x1 = self.conv1(x)
        x2 = self.conv2(x)
        x2 = torch.cat([self.conv2a(x2),self.conv2b(x2)], 1)
        x3 = self.conv3(x)
        x3 = torch.cat([self.conv2a(x3),self.conv2b(x3)], 1)
        x4 = self.conv4(x)
        x = torch.cat([x1,x2,x3,x4], 1)
        return x

In [26]:
class ReductionA(nn.Module):
    def __init__(self, in_channel):
        super(ReductionA, self).__init__()
        # 3x3 conv
        self.conv1 = BasicConv2d(in_channel, 384, kernel_size=3, stride=2)
        # 1x1 conv -> [3x3 conv]x2
        self.conv2 = nn.Sequential(
            BasicConv2d(in_channel, 64, kernel_size=1),
            BasicConv2d(64, 96, kernel_size=3, padding=1),
            BasicConv2d(96, 96, kernel_size=3, stride=2)
        )
        # 3x3 maxpool
        self.conv3 = nn.MaxPool2d(kernel_size=3, stride=2)
    
    def forward(self, x):
        x1 = self.conv1(x)
        x2 = self.conv2(x)
        x3 = self.conv3(x)
        x = torch.cat([x1,x2,x3], 1)
        return x

In [27]:
class ReductionB(nn.Module):
    def __init__(self, in_channel):
        super(ReductionB, self).__init__()
        # 1x1 conv -> 3x3 conv
        self.conv1 = nn.Sequential(
            BasicConv2d(in_channel, 192, kernel_size=1),
            BasicConv2d(192, 320, kernel_size=3, stride=2)
        )
        # 1x1 conv -> 1x7 conv -> 7x1 conv -> 3x3 conv
        self.conv2 = nn.Sequential(
            BasicConv2d(in_channel, 192, kernel_size=1),
            BasicConv2d(192, 192, kernel_size=(1,7), padding=(0,3)),
            BasicConv2d(192, 192, kernel_size=(7,1), padding=(3,0)),
            BasicConv2d(192, 192, kernel_size=3, stride=2)
        )
        # 3x3 maxpool
        self.conv3 = nn.MaxPool2d(kernel_size=3, stride=2)
    
    def forward(self, x):
        x1 = self.conv1(x)
        x2 = self.conv2(x)
        x3 = self.conv3(x)
        x = torch.cat([x1,x2,x3], 1)
        return x

In [28]:
class InceptionAux(nn.Module):
    def __init__(self, in_channel, num_classes):
        super(InceptionAux, self).__init__()
        self.pool0 = nn.AvgPool2d(kernel_size=5, stride=3)
        self.conv1 = BasicConv2d(in_channel, 128, kernel_size=1)
        self.conv2 = BasicConv2d(128, 768, kernel_size=5)
        self.conv2.stddev = 0.01
        self.fc = nn.Linear(768, num_classes)
        self.fc.stddev = 0.001
    
    def forward(self, x):
        x = self.pool0(x)
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [49]:
class InceptionV3(nn.Module):
    def __init__(self, num_classes=1000, aux_logits=False, transform_input=False):
        super(InceptionV3, self).__init__()
        self.aux_logits = aux_logits
        self.transform_input = transform_input
        self.Conv2d_1a_3x3 = BasicConv2d(3, 32, kernel_size=3, stride=2)
        self.Conv2d_2a_3x3 = BasicConv2d(32, 32, kernel_size=3)
        self.Conv2d_2b_3x3 = BasicConv2d(32, 64, kernel_size=3, padding=1)
        self.Conv2d_3b_1x1 = BasicConv2d(64, 80, kernel_size=1)
        self.Conv2d_4a_3x3 = BasicConv2d(80, 192, kernel_size=3)
        self.Mixed_5b = InceptionA(192, pool_feature=32)
        self.Mixed_5c = InceptionA(256, pool_feature=64)
        self.Mixed_5d = InceptionA(288, pool_feature=64)
        self.Mixed_6a = ReductionA(288)
        self.Mixed_6b = InceptionB(768, filter7x7=128)
        self.Mixed_6c = InceptionB(768, filter7x7=160)
        self.Mixed_6d = InceptionB(768, filter7x7=160)
        self.Mixed_6e = InceptionB(768, filter7x7=192)
        if aux_logits:
            self.AuxLogits = InceptionAux(768, num_classes)
        self.Mixed_7a = ReductionB(768)
        self.Mixed_7b = InceptionC(1280)
        self.Mixed_7c = InceptionC(2048)
        self.fc = nn.Linear(2048, num_classes)
    def forward(self, x):
        if self.transform_input: # 1
            x = x.clone()
            x[:, 0] = x[:, 0] * (0.229 / 0.5) + (0.485 - 0.5) / 0.5
            x[:, 1] = x[:, 1] * (0.224 / 0.5) + (0.456 - 0.5) / 0.5
            x[:, 2] = x[:, 2] * (0.225 / 0.5) + (0.406 - 0.5) / 0.5
        # 299 x 299 x 3
        x = self.Conv2d_1a_3x3(x)
        # 149 x 149 x 32
        x = self.Conv2d_2a_3x3(x)
        # 147 x 147 x 32
        x = self.Conv2d_2b_3x3(x)
        # 147 x 147 x 64
        x = F.max_pool2d(x, kernel_size=3, stride=2)
        # 73 x 73 x 64
        x = self.Conv2d_3b_1x1(x)
        # 73 x 73 x 80
        x = self.Conv2d_4a_3x3(x)
        # 71 x 71 x 192
        x = F.max_pool2d(x, kernel_size=3, stride=2)
        # 35 x 35 x 192
        x = self.Mixed_5b(x)
        # 35 x 35 x 256
        x = self.Mixed_5c(x)
        # 35 x 35 x 288
        x = self.Mixed_5d(x)
        # 35 x 35 x 288
        x = self.Mixed_6a(x)
        # 17 x 17 x 768
        x = self.Mixed_6b(x)
        # 17 x 17 x 768
        x = self.Mixed_6c(x)
        # 17 x 17 x 768
        x = self.Mixed_6d(x)
        # 17 x 17 x 768
        x = self.Mixed_6e(x)
        if self.training and self.aux_logits:
            aux = self.AuxLogits(x)
        # 17 x 17 x 768
        x = self.Mixed_7a(x)
        # 8 x 8 x 1280
        x = self.Mixed_7b(x)
        # 8 x 8 x 2048
        x = self.Mixed_7c(x)
        # 8 x 8 x 2048
        x = F.avg_pool2d(x, kernel_size=4)
        # 1 x 1 x 2048
        x = F.dropout(x, training=self.training)
        # 1 x 1 x 2048
        x = x.view(x.size(0), -1)
        # 2048
        x = self.fc(x)
        # 1000 (num_classes)
        if self.training and self.aux_logits:
            return x, aux
        return x

In [50]:
model = InceptionV3(num_classes=4)

In [51]:
model.to(device)
# 统计模型参数量以及其他指标
import torchsummary as summary
summary.summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 111, 111]             864
       BatchNorm2d-2         [-1, 32, 111, 111]              64
              ReLU-3         [-1, 32, 111, 111]               0
       BasicConv2d-4         [-1, 32, 111, 111]               0
            Conv2d-5         [-1, 32, 109, 109]           9,216
       BatchNorm2d-6         [-1, 32, 109, 109]              64
              ReLU-7         [-1, 32, 109, 109]               0
       BasicConv2d-8         [-1, 32, 109, 109]               0
            Conv2d-9         [-1, 64, 109, 109]          18,432
      BatchNorm2d-10         [-1, 64, 109, 109]             128
             ReLU-11         [-1, 64, 109, 109]               0
      BasicConv2d-12         [-1, 64, 109, 109]               0
           Conv2d-13           [-1, 80, 54, 54]           5,120
      BatchNorm2d-14           [-1, 80,

In [52]:
# 训练循环
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)  # 训练集的大小
    num_batches = len(dataloader)   # 批次数目, (size/batch_size，向上取整)

    train_loss, train_acc = 0, 0  # 初始化训练损失和正确率
    
    for X, y in dataloader:  # 获取图片及其标签
        X, y = X.to(device), y.to(device)
        
        # 计算预测误差
        pred = model(X)          # 网络输出
        loss = loss_fn(pred, y)  # 计算网络输出和真实值之间的差距，targets为真实值，计算二者差值即为损失
        
        # 反向传播
        optimizer.zero_grad()  # grad属性归零
        loss.backward()        # 反向传播
        optimizer.step()       # 每一步自动更新
        
        # 记录acc与loss
        train_acc  += (pred.argmax(1) == y).type(torch.float).sum().item()
        train_loss += loss.item()
            
    train_acc  /= size
    train_loss /= num_batches

    return train_acc, train_loss

In [53]:
def test (dataloader, model, loss_fn):
    size        = len(dataloader.dataset)  # 测试集的大小
    num_batches = len(dataloader)          # 批次数目, (size/batch_size，向上取整)
    test_loss, test_acc = 0, 0
    
    # 当不进行训练时，停止梯度更新，节省计算内存消耗
    with torch.no_grad():
        for imgs, target in dataloader:
            imgs, target = imgs.to(device), target.to(device)
            
            # 计算loss
            target_pred = model(imgs)
            loss        = loss_fn(target_pred, target)
            
            test_loss += loss.item()
            test_acc  += (target_pred.argmax(1) == target).type(torch.float).sum().item()

    test_acc  /= size
    test_loss /= num_batches

    return test_acc, test_loss

In [ ]:
import copy

optimizer  = torch.optim.Adam(model.parameters(), lr= 1e-4)
loss_fn    = nn.CrossEntropyLoss() # 创建损失函数

epochs     = 10

train_loss = []
train_acc  = []
test_loss  = []
test_acc   = []

best_acc = 0    # 设置一个最佳准确率，作为最佳模型的判别指标

for epoch in range(epochs):
    
    model.train()
    epoch_train_acc, epoch_train_loss = train(train_dl, model, loss_fn, optimizer)
    
    model.eval()
    epoch_test_acc, epoch_test_loss = test(test_dl, model, loss_fn)
    
    # 保存最佳模型到 best_model
    if epoch_test_acc > best_acc:
        best_acc   = epoch_test_acc
        best_model = copy.deepcopy(model)
    
    train_acc.append(epoch_train_acc)
    train_loss.append(epoch_train_loss)
    test_acc.append(epoch_test_acc)
    test_loss.append(epoch_test_loss)
    
    # 获取当前的学习率
    lr = optimizer.state_dict()['param_groups'][0]['lr']
    
    template = ('Epoch:{:2d}, Train_acc:{:.1f}%, Train_loss:{:.3f}, Test_acc:{:.1f}%, Test_loss:{:.3f}, Lr:{:.2E}')
    print(template.format(epoch+1, epoch_train_acc*100, epoch_train_loss, 
                          epoch_test_acc*100, epoch_test_loss, lr))
    
# 保存最佳模型到文件中
PATH = './best_model.pth'  # 保存的参数文件名
torch.save(model.state_dict(), PATH)

print('Done')

Epoch: 1, Train_acc:61.3%, Train_loss:0.967, Test_acc:54.4%, Test_loss:1.110, Lr:1.00E-04
Epoch: 2, Train_acc:73.7%, Train_loss:0.689, Test_acc:77.2%, Test_loss:0.683, Lr:1.00E-04
Epoch: 3, Train_acc:79.6%, Train_loss:0.547, Test_acc:75.4%, Test_loss:0.832, Lr:1.00E-04
Epoch: 4, Train_acc:85.6%, Train_loss:0.437, Test_acc:81.6%, Test_loss:0.661, Lr:1.00E-04
Epoch: 5, Train_acc:83.0%, Train_loss:0.435, Test_acc:78.9%, Test_loss:0.696, Lr:1.00E-04
Epoch: 6, Train_acc:86.9%, Train_loss:0.355, Test_acc:85.1%, Test_loss:0.375, Lr:1.00E-04
Epoch: 7, Train_acc:92.7%, Train_loss:0.239, Test_acc:87.7%, Test_loss:0.480, Lr:1.00E-04
Epoch: 8, Train_acc:88.7%, Train_loss:0.326, Test_acc:83.3%, Test_loss:0.458, Lr:1.00E-04
Epoch: 9, Train_acc:93.1%, Train_loss:0.218, Test_acc:86.0%, Test_loss:0.353, Lr:1.00E-04
